### [Text generation with an RNN](https://www.tensorflow.org/beta/tutorials/text/text_generation)

**Note:** Run in Google Colab with GPU support.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import os
import time

TensorFlow 2.x selected.


#### Download the dataset

In [0]:
file_path = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

#### Explore the data

In [0]:
text = open(file_path, 'rb').read().decode('utf-8')

In [4]:
print('Length: {}'.format(len(text)))

Length: 1115394


In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text))
print('Unique characters: {}'.format(len(vocab)))

Unique characters: 65


#### Text processing

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [0]:
text_as_int = np.array([char2idx[c] for c in text])

In [9]:
for char, _ in zip(char2idx, range(5)):
    print('{}: {}'.format(repr(char), char2idx[char]))

'\n': 0
' ': 1
'!': 2
'$': 3
'&': 4


In [10]:
print('{} ---> {}'.format(repr(text[:5]), text_as_int[:5]))

'First' ---> [18 47 56 57 58]


#### Prepare dataset

In [0]:
SEQ_LEN = 100
EXAMPLES_PER_EPOCH = len(text) // SEQ_LEN

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences = dataset.batch(SEQ_LEN+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [0]:
dataset = sequences.map(split_input_target)

#### Creating training batches

In [0]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

In [0]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

#### Define the model

In [0]:
VOCAB_SIZE = len(vocab)
EMB_DIM = 256
RNN_UNITS = 1024

In [0]:
def build_model(batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(RNN_UNITS, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(VOCAB_SIZE)
    ])
    return model

In [0]:
model = build_model(batch_size=BATCH_SIZE)

In [0]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam',
             loss=loss)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


#### Configure checkpoints

In [0]:
cp_dir = './training_checkpoints'
cp_prefix = os.path.join(cp_dir, 'ckpt_{epoch}')

In [0]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=cp_prefix, save_weights_only=True)

#### Training

In [0]:
EPOCHS = 30

In [27]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[cp_callback])

Epoch 1/30
172/172 [==============================] - 33s 191ms/step - loss: 2.5699
Epoch 2/30
172/172 [==============================] - 31s 182ms/step - loss: 1.8773
Epoch 3/30
172/172 [==============================] - 31s 180ms/step - loss: 1.6292
Epoch 4/30
172/172 [==============================] - 31s 182ms/step - loss: 1.5009
Epoch 5/30
172/172 [==============================] - 31s 178ms/step - loss: 1.4201
Epoch 6/30
172/172 [==============================] - 31s 182ms/step - loss: 1.3631
Epoch 7/30
172/172 [==============================] - 32s 186ms/step - loss: 1.3182
Epoch 8/30
172/172 [==============================] - 30s 174ms/step - loss: 1.2781
Epoch 9/30
172/172 [==============================] - 30s 177ms/step - loss: 1.2422
Epoch 10/30
172/172 [==============================] - 31s 181ms/step - loss: 1.2049
Epoch 11/30
172/172 [==============================] - 31s 180ms/step - loss: 1.1696
Epoch 12/30
172/172 [==============================] - 31s 182ms/step - lo

#### Generate text

To keep it simple, use a batch size of 1.

To run a model with different batch size, rebuild the model and restore the weights from the checkpoint.

In [0]:
model = build_model(batch_size=1)
model.load_weights(tf.train.latest_checkpoint(cp_dir))
model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
    NUM_GEN = 1000
    
    input_str = [char2idx[s] for s in start_string]
    input_str = tf.expand_dims(input_str, 0)
    
    text_generated = []
    
    temperature = 1.0
    
    model.reset_states()
    for i in range(NUM_GEN):
        preds = model(input_str)
        preds = tf.squeeze(preds, 0)
        
        preds = preds / temperature
        pred_id = tf.random.categorical(preds, num_samples=1)[-1, 0].numpy()
        
        input_str = tf.expand_dims([pred_id], 0)
        
        text_generated.append(idx2char[pred_id])

    return (start_string + ''.join(text_generated))

In [31]:
print(generate_text(model, start_string=u'ROMEO: '))

ROMEO: which triumphs with the rest were so tender one
O' the Lady Grey, who knew of Your flesh and mistress shall prove a boisterous coward,
As you guest wounded at, go virguius.

SICINIUS:
Have you a time lent privilege to prison?

GREMIO:
No, my good lord, my monown you are now a man.
Now, by Sadatory, you are retarning
Father be revenged on him thy kingly tido's will,
Waiting-gredients, and that not being made to fly to ush this and sunst thou little queen.
Good nurse, till he hath stopp'd the glory help to opent him.

Clown:
Waith!
To whom had he not?

Shepherd:
A good stout cheer:
Please you in plain, God hear the counterfeit steel most deadly belend,
In grief that every day infedient more.

Clown:
I'll ha' none: it is he took: in who beway of love,
'Had birds no woes for hand to die, and look not one of you.
Meantime, confide your brother Gloucester,
One with him, woman, indeed; for,
No girlt both! why, no; forbear thy sake: if thou
not, let it come to them absaff.

KING EDWARD 